<a href="https://colab.research.google.com/github/daycardoso/touche-code/blob/main/Avalia_arthur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [2]:
import pandas as pd

import torch

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from tqdm.notebook import tqdm

import numpy as np

import os


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd
import json
import os # Adicionado para construir os caminhos

# Carregar os Dados em Ingles
home = "/content/drive/MyDrive/Mestrado/DetectionOfHumanValuesInTexts/Colab_Experimentos/"

folder_treino = home + "training-english/"
folder_validacao = home + "validation-english/"
folder_teste = home + "test/"

# Json value-categories - Carregue o JSON uma única vez
caminho_json_valores = os.path.join(home, "value-categories.json")
with open(caminho_json_valores, 'r') as f:
    categorias_valores = json.load(f)

df_sentences_train = pd.read_csv(folder_treino + "sentences.tsv", sep='\t')
df_labels_train = pd.read_csv(folder_treino + "labels.tsv", sep='\t')

df_sentences_valid = pd.read_csv(folder_validacao + "sentences.tsv", sep='\t')
df_labels_valid = pd.read_csv(folder_validacao + "labels.tsv", sep='\t')

df_sentences_test = pd.read_csv(folder_teste + "sentences.tsv", sep='\t')
df_labels_test = pd.read_csv(folder_teste + "labels.tsv", sep='\t')

# Json value-categories
categorias_valores = json.load(open(home + "value-categories.json"))

df_sentences_train.head()
df_sentences_train.describe()

,Sentence-ID
count,44758.000000
mean,16.530475
std,11.499004
min,1.000000
25%,7.000000
50%,15.000000
75%,23.000000
max,82.000000


In [7]:
sentences_df = df_sentences_test
print(sentences_df.head())

  Text-ID  Sentence-ID                                               Text
0  BG_001            1  Бежанка дойде от Украйна, у нас й вгорчават ощ...
1  BG_001            2  Бежанка от Украйна разкри абсурдите на българс...
2  BG_001            3  Ксения Бутова е на наша територия от няколко д...
3  BG_001            4  Тя пристига от Варна, но въпреки че там има ми...
4  BG_001            5  "Да, знаят за вход с виза тип "Д", но там не с...


In [8]:
MODEL_NAME = 'SotirisLegkas/multi-head-xlm-xl-tokens-38'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Usando dispositivo: {device}')

config = AutoConfig.from_pretrained(MODEL_NAME, num_labels=38)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME
                                                           , config=config)

model.to(device)

model.eval()

Usando dispositivo: cuda


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of XLMRobertaXLForSequenceClassification were not initialized from the model checkpoint at SotirisLegkas/multi-head-xlm-xl-tokens-38 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLMRobertaXLForSequenceClassification(
  (roberta): XLMRobertaXLModel(
    (embeddings): XLMRobertaXLEmbeddings(
      (word_embeddings): Embedding(250041, 2560, padding_idx=1)
      (position_embeddings): Embedding(514, 2560, padding_idx=1)
      (token_type_embeddings): Embedding(1, 2560)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaXLEncoder(
      (layer): ModuleList(
        (0-35): 36 x XLMRobertaXLLayer(
          (attention): XLMRobertaXLAttention(
            (self_attn_layer_norm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (self): XLMRobertaXLSdpaSelfAttention(
              (query): Linear(in_features=2560, out_features=2560, bias=True)
              (key): Linear(in_features=2560, out_features=2560, bias=True)
              (value): Linear(in_features=2560, out_features=2560, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaXLSelfOutput(
              (de

In [9]:
def predict_batch(texts):


    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=512)

    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():

        outputs = model(**inputs)

    logits = outputs.logits

    probabilities = torch.sigmoid(logits)

    return probabilities.cpu().numpy()



batch_size = 16

all_predictions = []



texts = sentences_df['Text'].tolist()



for i in tqdm(range(0, len(texts), batch_size)):

    batch_texts = texts[i:i+batch_size]

    batch_preds = predict_batch(batch_texts)

    all_predictions.extend(batch_preds)

  0%|          | 0/911 [00:00<?, ?it/s]

In [10]:
# Nomes dos valores e colunas, conforme definido no script do avaliador

availableValues = ["Self-direction: thought", "Self-direction: action", "Stimulation", "Hedonism", "Achievement", "Power: dominance", "Power: resources", "Face", "Security: personal", "Security: societal", "Tradition", "Conformity: rules", "Conformity: interpersonal", "Humility", "Benevolence: caring", "Benevolence: dependability", "Universalism: concern", "Universalism: nature", "Universalism: tolerance"]

value_columns = [value + postfix for value in availableValues for postfix in [' attained', ' constrained']]



# Criar o DataFrame de resultados

predictions_df = pd.DataFrame(all_predictions, columns=value_columns)



# Combinar com os IDs das sentenÃ§as

run_df = pd.concat([sentences_df[['Text-ID', 'Sentence-ID']], predictions_df], axis=1)



# Garantir que o diretÃ³rio de saÃ­da exista

output_dir = home + 'run'

if not os.path.exists(output_dir):

    os.makedirs(output_dir)



# Salvar o arquivo run.tsv

output_path = os.path.join(output_dir, 'run.tsv')

run_df.to_csv(output_path, sep='	', index=False, float_format='%.4f')



print(f"Arquivo 'run.tsv' salvo em: {os.path.abspath(output_path)}")

print("Amostra do arquivo de saÃ­da:")

print(run_df.head())

Arquivo 'run.tsv' salvo em: /content/drive/MyDrive/Mestrado/DetectionOfHumanValuesInTexts/Colab_Experimentos/run/run.tsv
Amostra do arquivo de saÃ­da:
  Text-ID  Sentence-ID  Self-direction: thought attained  \
0  BG_001            1                          0.502780   
1  BG_001            2                          0.502272   
2  BG_001            3                          0.502299   
3  BG_001            4                          0.503139   
4  BG_001            5                          0.501993   

   Self-direction: thought constrained  Self-direction: action attained  \
0                             0.518973                         0.484993   
1                             0.518876                         0.485162   
2                             0.519432                         0.485202   
3                             0.518598                         0.484536   
4                             0.517461                         0.484916   

   Self-direction: action constrained